In [1]:
import csv
import pandas as pd
import numpy as np
import torch
from torch import nn, Tensor
from torch.utils.data import IterableDataset
import random
import os

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
v1_csv_file = r"./data/v1_a.csv"
df = pd.read_csv(v1_csv_file)
df.drop(columns="Field Index", inplace=True)

In [4]:
df.head(3)

,SENTINEL2/13,SENTINEL2/16,SERENUS_GEOTIFF/1,SERENUS_GEOTIFF/6,SENTINEL2/ndti,pixel_x,SERENUS_GEOTIFF/7,SENTINEL2/2,SERENUS_GEOTIFF/14,SERENUS_GEOTIFF/15,SERENUS_GEOTIFF/3,SENTINEL2/8,CLOUD_MASK_PNG/time_secs,tile_height,SERENUS_GEOTIFF/16,SERENUS_GEOTIFF/ndti,SENTINEL2/5,SENTINEL2/time_secs,tile_y,SENTINEL2/15,SENTINEL2/savi,SERENUS_GEOTIFF/10,SENTINEL2/9,SENTINEL2/7,SERENUS_GEOTIFF/11,SENTINEL2/image_path,SENTINEL2/1,SERENUS_GEOTIFF/13,SERENUS_GEOTIFF/ndvi,tile_width,SERENUS_GEOTIFF/5,field_id,SERENUS_GEOTIFF/savi,CLOUD_MASK_PNG/image_path,tile_x,SENTINEL2/4,SENTINEL2/17,county_fips_code,golden_label,SENTINEL2/10,SERENUS_GEOTIFF/time_secs,SENTINEL2/6,zoom_level,SENTINEL2/12,CLOUD_MASK_PNG/1,SERENUS_GEOTIFF/9,SENTINEL2/14,SERENUS_GEOTIFF/2,SENTINEL2/ndvi,SENTINEL2/3,SENTINEL2/11,pixel_y,SERENUS_GEOTIFF/8,SERENUS_GEOTIFF/image_path,SERENUS_GEOTIFF/4,SERENUS_GEOTIFF/12
0,"[1494, 1932, 1029, 2609, 1880, 2016, 1176, 325...","[1024, 1024, 1024, 1024, 1024, 1024, 0, 0, 102...","[1458, 1460, 1451, 1451, 1450, 1418, 1417, 142...","[1593, 1594, 1565, 1566, 1564, 1470, 1464, 146...","[-0.9757575988769531, -0.4324128031730652, -0....",[225],"[1739, 1738, 1728, 1728, 1725, 1617, 1610, 160...","[6143, 5683, 4452, 7170, 7531, 7041, 8526, 809...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1151, 1147, 1118, 1138, 1133, 1095, 1029, 103...","[5517, 5419, 4087, 7063, 7182, 6629, 8512, 787...","[1577923200, 1578355200, 1578787200, 157921920...",[705],"[41, 38, 36, 29, 22, 20, 17, 23, 21, 27, 22, 1...","[-0.9389830231666565, -0.939830482006073, -0.9...","[5838, 5501, 4258, 7178, 7346, 6856, 8993, 858...","[1577923200, 1578355200, 1578787200, 157921920...",[1488],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.06421815603971481, 0.08848532289266586, 0.0...","[1198, 1196, 1195, 1197, 1195, 1151, 1146, 114...","[5842, 5582, 4278, 7326, 7660, 7011, 8889, 806...","[5960, 5604, 4305, 7328, 7640, 7021, 9005, 837...","[72, 71, 71, 71, 70, 68, 67, 68, 67, 68, 67, 6...","[b'2020-01-02.tif', b'2020-01-07.tif', b'2020-...","[6363, 6005, 4758, 7421, 7738, 7296, 8634, 804...","[1590, 1590, 1599, 1604, 1605, 1499, 1490, 150...","[0.2034602016210556, 0.20485268533229828, 0.21...",[980],"[1455, 1456, 1417, 1419, 1416, 1308, 1301, 130...",[b'1402'],"[0.29049816727638245, 0.2924862504005432, 0.30...","[b'2020-01-02.png', b'2020-01-07.png', b'2020-...",[1000],"[5794, 5385, 4123, 7041, 7313, 6797, 8720, 834...","[65535, 65535, 65535, 65535, 65535, 65535, 655...",[27109],[b'corn'],"[1488, 3585, 2061, 5635, 3848, 2602, 3818, 382...","[1577923200, 1578355200, 1578787200, 157921920...","[5874, 5504, 4300, 7209, 7484, 6918, 9045, 859...",[12],"[1956, 1971, 1029, 2548, 1756, 2818, 1165, 341...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1979, 1978, 1937, 1939, 1938, 1862, 1854, 185...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1278, 1290, 1277, 1288, 1284, 1283, 1196, 119...","[0.044972386211156845, 0.06196702644228935, 0....","[5447, 4950, 3897, 6618, 6777, 6333, 7934, 761...","[24, 781, 259, 3199, 821, 22, 19, 33, 85, 200,...",[392],"[1847, 1821, 1834, 1832, 1828, 1805, 1689, 168...","[b'2020-01-02.tif', b'2020-01-07.tif', b'2020-...","[1298, 1279, 1282, 1309, 1305, 1259, 1170, 117...","[2288, 2289, 2299, 2308, 2310, 2178, 2169, 218..."
1,"[4206, 3370, 1363, 2079, 2924, 1472, 180, 1118...","[1024, 1024, 0, 1024, 1024, 1024, 0, 0, 1024, ...","[1589, 1872, 1904, 1907, 1871, 1624, 1620, 161...","[1121, 2188, 2386, 1698, 1577, 1554, 1404, 170...","[-0.984599769115448, -0.9402085542678833, -0.9...",[800],"[1187, 2574, 2905, 1765, 1920, 1906, 1757, 204...","[5388, 4972, 3307, 6163, 7032, 6872, 5474, 325...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1040, 2942, 1511, 1257, 1299, 1219, 1115, 165...","[5403, 5158, 3750, 5867, 6629, 6430, 5343, 358...","[1578096000, 1578528000, 1578960000, 157939200...",[741],"[6, 1419, 1393,

In [5]:
columns_to_drop = ['pixel_x', 'tile_height', 'CLOUD_MASK_PNG/time_secs', 'SENTINEL2/time_secs','tile_y', 
                   'SENTINEL2/image_path', 'tile_width', 'field_id', 'CLOUD_MASK_PNG/image_path', 'tile_x', 
                   'county_fips_code', 'zoom_level', 'CLOUD_MASK_PNG/1', 'pixel_y', 'SERENUS_GEOTIFF/image_path', 
                   'SERENUS_GEOTIFF/time_secs']

In [6]:
len(columns_to_drop)

16

In [7]:
df.columns

Index(['SENTINEL2/13', 'SENTINEL2/16', 'SERENUS_GEOTIFF/1',
       'SERENUS_GEOTIFF/6', 'SENTINEL2/ndti', 'pixel_x', 'SERENUS_GEOTIFF/7',
       'SENTINEL2/2', 'SERENUS_GEOTIFF/14', 'SERENUS_GEOTIFF/15',
       'SERENUS_GEOTIFF/3', 'SENTINEL2/8', 'CLOUD_MASK_PNG/time_secs',
       'tile_height', 'SERENUS_GEOTIFF/16', 'SERENUS_GEOTIFF/ndti',
       'SENTINEL2/5', 'SENTINEL2/time_secs', 'tile_y', 'SENTINEL2/15',
       'SENTINEL2/savi', 'SERENUS_GEOTIFF/10', 'SENTINEL2/9', 'SENTINEL2/7',
       'SERENUS_GEOTIFF/11', 'SENTINEL2/image_path', 'SENTINEL2/1',
       'SERENUS_GEOTIFF/13', 'SERENUS_GEOTIFF/ndvi', 'tile_width',
       'SERENUS_GEOTIFF/5', 'field_id', 'SERENUS_GEOTIFF/savi',
       'CLOUD_MASK_PNG/image_path', 'tile_x', 'SENTINEL2/4', 'SENTINEL2/17',
       'county_fips_code', 'golden_label', 'SENTINEL2/10',
       'SERENUS_GEOTIFF/time_secs', 'SENTINEL2/6', 'zoom_level',
       'SENTINEL2/12', 'CLOUD_MASK_PNG/1', 'SERENUS_GEOTIFF/9', 'SENTINEL2/14',
       'SERENUS_GEOTIFF/2', '

In [8]:
df.drop(columns=columns_to_drop, inplace=True)

In [9]:
df.head(3)

,SENTINEL2/13,SENTINEL2/16,SERENUS_GEOTIFF/1,SERENUS_GEOTIFF/6,SENTINEL2/ndti,SERENUS_GEOTIFF/7,SENTINEL2/2,SERENUS_GEOTIFF/14,SERENUS_GEOTIFF/15,SERENUS_GEOTIFF/3,SENTINEL2/8,SERENUS_GEOTIFF/16,SERENUS_GEOTIFF/ndti,SENTINEL2/5,SENTINEL2/15,SENTINEL2/savi,SERENUS_GEOTIFF/10,SENTINEL2/9,SENTINEL2/7,SERENUS_GEOTIFF/11,SENTINEL2/1,SERENUS_GEOTIFF/13,SERENUS_GEOTIFF/ndvi,SERENUS_GEOTIFF/5,SERENUS_GEOTIFF/savi,SENTINEL2/4,SENTINEL2/17,golden_label,SENTINEL2/10,SENTINEL2/6,SENTINEL2/12,SERENUS_GEOTIFF/9,SENTINEL2/14,SERENUS_GEOTIFF/2,SENTINEL2/ndvi,SENTINEL2/3,SENTINEL2/11,SERENUS_GEOTIFF/8,SERENUS_GEOTIFF/4,SERENUS_GEOTIFF/12
0,"[1494, 1932, 1029, 2609, 1880, 2016, 1176, 325...","[1024, 1024, 1024, 1024, 1024, 1024, 0, 0, 102...","[1458, 1460, 1451, 1451, 1450, 1418, 1417, 142...","[1593, 1594, 1565, 1566, 1564, 1470, 1464, 146...","[-0.9757575988769531, -0.4324128031730652, -0....","[1739, 1738, 1728, 1728, 1725, 1617, 1610, 160...","[6143, 5683, 4452, 7170, 7531, 7041, 8526, 809...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1151, 1147, 1118, 1138, 1133, 1095, 1029, 103...","[5517, 5419, 4087, 7063, 7182, 6629, 8512, 787...","[41, 38, 36, 29, 22, 20, 17, 23, 21, 27, 22, 1...","[-0.9389830231666565, -0.939830482006073, -0.9...","[5838, 5501, 4258, 7178, 7346, 6856, 8993, 858...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.06421815603971481, 0.08848532289266586, 0.0...","[1198, 1196, 1195, 1197, 1195, 1151, 1146, 114...","[5842, 5582, 4278, 7326, 7660, 7011, 8889, 806...","[5960, 5604, 4305, 7328, 7640, 7021, 9005, 837...","[72, 71, 71, 71, 70, 68, 67, 68, 67, 68, 67, 6...","[6363, 6005, 4758, 7421, 7738, 7296, 8634, 804...","[1590, 1590, 1599, 1604, 1605, 1499, 1490, 150...","[0.2034602016210556, 0.20485268533229828, 0.21...","[1455, 1456, 1417, 1419, 1416, 1308, 1301, 130...","[0.29049816727638245, 0.2924862504005432, 0.30...","[5794, 5385, 4123, 7041, 7313, 6797, 8720, 834...","[65535, 65535, 65535, 65535, 65535, 65535, 655...",[b'corn'],"[1488, 3585, 2061, 5635, 3848, 2602, 3818, 382...","[5874, 5504, 4300, 7209, 7484, 6918, 9045, 859...","[1956, 1971, 1029, 2548, 1756, 2818, 1165, 341...","[1979, 1978, 1937, 1939, 1938, 1862, 1854, 185...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1278, 1290, 1277, 1288, 1284, 1283, 1196, 119...","[0.044972386211156845, 0.06196702644228935, 0....","[5447, 4950, 3897, 6618, 6777, 6333, 7934, 761...","[24, 781, 259, 3199, 821, 22, 19, 33, 85, 200,...","[1847, 1821, 1834, 1832, 1828, 1805, 1689, 168...","[1298, 1279, 1282, 1309, 1305, 1259, 1170, 117...","[2288, 2289, 2299, 2308, 2310, 2178, 2169, 218..."
1,"[4206, 3370, 1363, 2079, 2924, 1472, 180, 1118...","[1024, 1024, 0, 1024, 1024, 1024, 0, 0, 1024, ...","[1589, 1872, 1904, 1907, 1871, 1624, 1620, 161...","[1121, 2188, 2386, 1698, 1577, 1554, 1404, 170...","[-0.984599769115448, -0.9402085542678833, -0.9...","[1187, 2574, 2905, 1765, 1920, 1906, 1757, 204...","[5388, 4972, 3307, 6163, 7032, 6872, 5474, 325...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1040, 2942, 1511, 1257, 1299, 1219, 1115, 165...","[5403, 5158, 3750, 5867, 6629, 6430, 5343, 358...","[6, 1419, 1393, 1364, 1345, 1317, 1291, 1281, ...","[-0.9636576771736145, -0.9060665369033813, -0....","[5373, 4184, 3505, 6138, 6712, 6628, 5707, 358...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.11580117046833038, -0.0443173311650753, 0.1...","[749, 859, 1043, 1133, 1129, 969, 843, 890, 99...","[5821, 4032, 4009, 6028, 7132, 6854, 5657, 388...","[5659, 3965, 3727, 6196, 7077, 6890, 5831, 402...","[31, 72, 68, 73, 74, 66, 59, 63, 39, 71, 41, 2...","[5581, 3875, 3305, 6331, 7438, 7210, 5397, 380...","[1306, 872, 935, 1036, 1089, 1167, 1316, 1321,...","[0.0660080835223198, -0.06671500951051712, 0.3...","[1063, 2060, 2207, 1395, 1327, 1383, 1295, 153...","[0.0942414328455925, -0.09526164084672928, 0.4...","[5276, 4214, 3279, 6000, 669

In [10]:
df.shape

(1240, 40)

In [11]:
columns = df.columns

In [12]:
sentinel2_cols = [col for col in columns if col.startswith("SENTINEL2")]
serenus_geotiff_cols = [col for col in columns if col.startswith("SERENUS_GEOTIFF")]


In [13]:
len(sentinel2_cols)

20

In [14]:
len(serenus_geotiff_cols)

19

In [15]:
def replace_char(a):
    b = ['[', ']']
    for char in b:
        a = a.replace(char, "")
    a = [float(k) for k in a.split(",")]
    return a

In [16]:
feature_cols = sentinel2_cols + serenus_geotiff_cols

In [17]:
for column in feature_cols:
    df[column] = [replace_char(df[column][i]) for i in range(df.shape[0])]

In [18]:
df.head(3)

,SENTINEL2/13,SENTINEL2/16,SERENUS_GEOTIFF/1,SERENUS_GEOTIFF/6,SENTINEL2/ndti,SERENUS_GEOTIFF/7,SENTINEL2/2,SERENUS_GEOTIFF/14,SERENUS_GEOTIFF/15,SERENUS_GEOTIFF/3,SENTINEL2/8,SERENUS_GEOTIFF/16,SERENUS_GEOTIFF/ndti,SENTINEL2/5,SENTINEL2/15,SENTINEL2/savi,SERENUS_GEOTIFF/10,SENTINEL2/9,SENTINEL2/7,SERENUS_GEOTIFF/11,SENTINEL2/1,SERENUS_GEOTIFF/13,SERENUS_GEOTIFF/ndvi,SERENUS_GEOTIFF/5,SERENUS_GEOTIFF/savi,SENTINEL2/4,SENTINEL2/17,golden_label,SENTINEL2/10,SENTINEL2/6,SENTINEL2/12,SERENUS_GEOTIFF/9,SENTINEL2/14,SERENUS_GEOTIFF/2,SENTINEL2/ndvi,SENTINEL2/3,SENTINEL2/11,SERENUS_GEOTIFF/8,SERENUS_GEOTIFF/4,SERENUS_GEOTIFF/12
0,"[1494.0, 1932.0, 1029.0, 2609.0, 1880.0, 2016....","[1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024....","[1458.0, 1460.0, 1451.0, 1451.0, 1450.0, 1418....","[1593.0, 1594.0, 1565.0, 1566.0, 1564.0, 1470....","[-0.9757575988769531, -0.4324128031730652, -0....","[1739.0, 1738.0, 1728.0, 1728.0, 1725.0, 1617....","[6143.0, 5683.0, 4452.0, 7170.0, 7531.0, 7041....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1151.0, 1147.0, 1118.0, 1138.0, 1133.0, 1095....","[5517.0, 5419.0, 4087.0, 7063.0, 7182.0, 6629....","[41.0, 38.0, 36.0, 29.0, 22.0, 20.0, 17.0, 23....","[-0.9389830231666565, -0.939830482006073, -0.9...","[5838.0, 5501.0, 4258.0, 7178.0, 7346.0, 6856....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.06421815603971481, 0.08848532289266586, 0.0...","[1198.0, 1196.0, 1195.0, 1197.0, 1195.0, 1151....","[5842.0, 5582.0, 4278.0, 7326.0, 7660.0, 7011....","[5960.0, 5604.0, 4305.0, 7328.0, 7640.0, 7021....","[72.0, 71.0, 71.0, 71.0, 70.0, 68.0, 67.0, 68....","[6363.0, 6005.0, 4758.0, 7421.0, 7738.0, 7296....","[1590.0, 1590.0, 1599.0, 1604.0, 1605.0, 1499....","[0.2034602016210556, 0.20485268533229828, 0.21...","[1455.0, 1456.0, 1417.0, 1419.0, 1416.0, 1308....","[0.29049816727638245, 0.2924862504005432, 0.30...","[5794.0, 5385.0, 4123.0, 7041.0, 7313.0, 6797....","[65535.0, 65535.0, 65535.0, 65535.0, 65535.0, ...",[b'corn'],"[1488.0, 3585.0, 2061.0, 5635.0, 3848.0, 2602....","[5874.0, 5504.0, 4300.0, 7209.0, 7484.0, 6918....","[1956.0, 1971.0, 1029.0, 2548.0, 1756.0, 2818....","[1979.0, 1978.0, 1937.0, 1939.0, 1938.0, 1862....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1278.0, 1290.0, 1277.0, 1288.0, 1284.0, 1283....","[0.044972386211156845, 0.06196702644228935, 0....","[5447.0, 4950.0, 3897.0, 6618.0, 6777.0, 6333....","[24.0, 781.0, 259.0, 3199.0, 821.0, 22.0, 19.0...","[1847.0, 1821.0, 1834.0, 1832.0, 1828.0, 1805....","[1298.0, 1279.0, 1282.0, 1309.0, 1305.0, 1259....","[2288.0, 2289.0, 2299.0, 2308.0, 2310.0, 2178...."
1,"[4206.0, 3370.0, 1363.0, 2079.0, 2924.0, 1472....","[1024.0, 1024.0, 0.0, 1024.0, 1024.0, 1024.0, ...","[1589.0, 1872.0, 1904.0, 1907.0, 1871.0, 1624....","[1121.0, 2188.0, 2386.0, 1698.0, 1577.0, 1554....","[-0.984599769115448, -0.9402085542678833, -0.9...","[1187.0, 2574.0, 2905.0, 1765.0, 1920.0, 1906....","[5388.0, 4972.0, 3307.0, 6163.0, 7032.0, 6872....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1040.0, 2942.0, 1511.0, 1257.0, 1299.0, 1219....","[5403.0, 5158.0, 3750.0, 5867.0, 6629.0, 6430....","[6.0, 1419.0, 1393.0, 1364.0, 1345.0, 1317.0, ...","[-0.9636576771736145, -0.9060665369033813, -0....","[5373.0, 4184.0, 3505.0, 6138.0, 6712.0, 6628....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.11580117046833038, -0.0443173311650753, 0.1...","[749.0, 859.0, 1043.0, 1133.0, 1129.0, 969.0, ...","[5821.0, 4032.0, 4009.0, 6028.0, 7132.0, 6854....","[5659.0, 3965.0, 3727.0, 6196.0, 7077.0, 6890....","[31.0, 72.0, 68.0, 73.0, 74.0, 66.0, 59.0, 63....","[5581.0, 3875.0, 3305.0, 6331.0, 7438.0, 7210....","[1306.0, 872.0, 935.0, 1036.0, 1089.0, 1167.0,...","[0.0660080835223198, -0.06671500951051712, 0.3...","[1063.0, 2060.0, 2207.0, 1395.0, 1327.0, 1383....","[0.0942414328455925, -0.09526164084672928, 0.4...","[5276.0, 4214.0, 3279.0, 600

In [19]:
for column in feature_cols:
    if column.endswith("ndti") or column.endswith("savi") or column.endswith("ndvi"):
        continue
    else:
        df[column] = [[k/10000 for k in df[column][i]] for i in range(df.shape[0])]

In [20]:
df["SENTINEL2/17"]

0       [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
1       [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
2       [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
3       [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
4       [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
                              ...                        
1235    [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
1236    [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
1237    [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
1238    [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
1239    [6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...
Name: SENTINEL2/17, Length: 1240, dtype: object

In [21]:
df.head(3)

,SENTINEL2/13,SENTINEL2/16,SERENUS_GEOTIFF/1,SERENUS_GEOTIFF/6,SENTINEL2/ndti,SERENUS_GEOTIFF/7,SENTINEL2/2,SERENUS_GEOTIFF/14,SERENUS_GEOTIFF/15,SERENUS_GEOTIFF/3,SENTINEL2/8,SERENUS_GEOTIFF/16,SERENUS_GEOTIFF/ndti,SENTINEL2/5,SENTINEL2/15,SENTINEL2/savi,SERENUS_GEOTIFF/10,SENTINEL2/9,SENTINEL2/7,SERENUS_GEOTIFF/11,SENTINEL2/1,SERENUS_GEOTIFF/13,SERENUS_GEOTIFF/ndvi,SERENUS_GEOTIFF/5,SERENUS_GEOTIFF/savi,SENTINEL2/4,SENTINEL2/17,golden_label,SENTINEL2/10,SENTINEL2/6,SENTINEL2/12,SERENUS_GEOTIFF/9,SENTINEL2/14,SERENUS_GEOTIFF/2,SENTINEL2/ndvi,SENTINEL2/3,SENTINEL2/11,SERENUS_GEOTIFF/8,SERENUS_GEOTIFF/4,SERENUS_GEOTIFF/12
0,"[0.1494, 0.1932, 0.1029, 0.2609, 0.188, 0.2016...","[0.1024, 0.1024, 0.1024, 0.1024, 0.1024, 0.102...","[0.1458, 0.146, 0.1451, 0.1451, 0.145, 0.1418,...","[0.1593, 0.1594, 0.1565, 0.1566, 0.1564, 0.147...","[-0.9757575988769531, -0.4324128031730652, -0....","[0.1739, 0.1738, 0.1728, 0.1728, 0.1725, 0.161...","[0.6143, 0.5683, 0.4452, 0.717, 0.7531, 0.7041...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1151, 0.1147, 0.1118, 0.1138, 0.1133, 0.109...","[0.5517, 0.5419, 0.4087, 0.7063, 0.7182, 0.662...","[0.0041, 0.0038, 0.0036, 0.0029, 0.0022, 0.002...","[-0.9389830231666565, -0.939830482006073, -0.9...","[0.5838, 0.5501, 0.4258, 0.7178, 0.7346, 0.685...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.06421815603971481, 0.08848532289266586, 0.0...","[0.1198, 0.1196, 0.1195, 0.1197, 0.1195, 0.115...","[0.5842, 0.5582, 0.4278, 0.7326, 0.766, 0.7011...","[0.596, 0.5604, 0.4305, 0.7328, 0.764, 0.7021,...","[0.0072, 0.0071, 0.0071, 0.0071, 0.007, 0.0068...","[0.6363, 0.6005, 0.4758, 0.7421, 0.7738, 0.729...","[0.159, 0.159, 0.1599, 0.1604, 0.1605, 0.1499,...","[0.2034602016210556, 0.20485268533229828, 0.21...","[0.1455, 0.1456, 0.1417, 0.1419, 0.1416, 0.130...","[0.29049816727638245, 0.2924862504005432, 0.30...","[0.5794, 0.5385, 0.4123, 0.7041, 0.7313, 0.679...","[6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...",[b'corn'],"[0.1488, 0.3585, 0.2061, 0.5635, 0.3848, 0.260...","[0.5874, 0.5504, 0.43, 0.7209, 0.7484, 0.6918,...","[0.1956, 0.1971, 0.1029, 0.2548, 0.1756, 0.281...","[0.1979, 0.1978, 0.1937, 0.1939, 0.1938, 0.186...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1278, 0.129, 0.1277, 0.1288, 0.1284, 0.1283...","[0.044972386211156845, 0.06196702644228935, 0....","[0.5447, 0.495, 0.3897, 0.6618, 0.6777, 0.6333...","[0.0024, 0.0781, 0.0259, 0.3199, 0.0821, 0.002...","[0.1847, 0.1821, 0.1834, 0.1832, 0.1828, 0.180...","[0.1298, 0.1279, 0.1282, 0.1309, 0.1305, 0.125...","[0.2288, 0.2289, 0.2299, 0.2308, 0.231, 0.2178..."
1,"[0.4206, 0.337, 0.1363, 0.2079, 0.2924, 0.1472...","[0.1024, 0.1024, 0.0, 0.1024, 0.1024, 0.1024, ...","[0.1589, 0.1872, 0.1904, 0.1907, 0.1871, 0.162...","[0.1121, 0.2188, 0.2386, 0.1698, 0.1577, 0.155...","[-0.984599769115448, -0.9402085542678833, -0.9...","[0.1187, 0.2574, 0.2905, 0.1765, 0.192, 0.1906...","[0.5388, 0.4972, 0.3307, 0.6163, 0.7032, 0.687...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.104, 0.2942, 0.1511, 0.1257, 0.1299, 0.1219...","[0.5403, 0.5158, 0.375, 0.5867, 0.6629, 0.643,...","[0.0006, 0.1419, 0.1393, 0.1364, 0.1345, 0.131...","[-0.9636576771736145, -0.9060665369033813, -0....","[0.5373, 0.4184, 0.3505, 0.6138, 0.6712, 0.662...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.11580117046833038, -0.0443173311650753, 0.1...","[0.0749, 0.0859, 0.1043, 0.1133, 0.1129, 0.096...","[0.5821, 0.4032, 0.4009, 0.6028, 0.7132, 0.685...","[0.5659, 0.3965, 0.3727, 0.6196, 0.7077, 0.689...","[0.0031, 0.0072, 0.0068, 0.0073, 0.0074, 0.006...","[0.5581, 0.3875, 0.3305, 0.6331, 0.7438, 0.721...","[0.1306, 0.0872, 0.0935, 0.1036, 0.1089, 0.116...","[0.0660080835223198, -0.06671500951051712, 0.3...","[0.1063, 0.206, 0.2207, 0.1395, 0.1327, 0.1383...","[0.0942414328455925, -0.09526164084672928, 0.4...","[0.5276, 0.4214, 0.3279, 0.6

In [22]:
df.shape

(1240, 40)

In [23]:
sentinel2_cols[:2]

['SENTINEL2/13', 'SENTINEL2/16']

In [24]:
df[sentinel2_cols[:2]].head(3)

,SENTINEL2/13,SENTINEL2/16
0,"[0.1494, 0.1932, 0.1029, 0.2609, 0.188, 0.2016...","[0.1024, 0.1024, 0.1024, 0.1024, 0.1024, 0.102..."
1,"[0.4206, 0.337, 0.1363, 0.2079, 0.2924, 0.1472...","[0.1024, 0.1024, 0.0, 0.1024, 0.1024, 0.1024, ..."
2,"[0.2912, 0.2297, 0.2478, 0.2295, 0.1891, 0.279...","[0.1024, 0.0, 0.1024, 0.1024, 0.0, 0.1024, 0.1..."


In [25]:
len(df[sentinel2_cols[:2]].sum(axis=1)[0])

150

In [26]:
df['golden_label'].nunique()

12

In [27]:
df['golden_label'].unique()

array(["[b'corn']", "[b'others']", "[b'fallow']", "[b'drybean']",
       "[b'soybean']", "[b'alfalfa']", "[b'cotton']", "[b'grassland']",
       "[b'wheat']", "[b'barley']", "[b'canola']", "[b'sorghum']"],
      dtype=object)

In [28]:
df["golden_label"].value_counts()

[b'grassland']    115
[b'wheat']        114
[b'soybean']      109
[b'cotton']       106
[b'fallow']       106
[b'corn']         106
[b'sorghum']      103
[b'alfalfa']      100
[b'canola']        98
[b'others']        97
[b'barley']        95
[b'drybean']       91
Name: golden_label, dtype: int64

In [33]:
df["feature"] = df[feature_cols].sum(axis=1)

In [34]:
df.head(3)

,SENTINEL2/13,SENTINEL2/16,SERENUS_GEOTIFF/1,SERENUS_GEOTIFF/6,SENTINEL2/ndti,SERENUS_GEOTIFF/7,SENTINEL2/2,SERENUS_GEOTIFF/14,SERENUS_GEOTIFF/15,SERENUS_GEOTIFF/3,SENTINEL2/8,SERENUS_GEOTIFF/16,SERENUS_GEOTIFF/ndti,SENTINEL2/5,SENTINEL2/15,SENTINEL2/savi,SERENUS_GEOTIFF/10,SENTINEL2/9,SENTINEL2/7,SERENUS_GEOTIFF/11,SENTINEL2/1,SERENUS_GEOTIFF/13,SERENUS_GEOTIFF/ndvi,SERENUS_GEOTIFF/5,SERENUS_GEOTIFF/savi,SENTINEL2/4,SENTINEL2/17,golden_label,SENTINEL2/10,SENTINEL2/6,SENTINEL2/12,SERENUS_GEOTIFF/9,SENTINEL2/14,SERENUS_GEOTIFF/2,SENTINEL2/ndvi,SENTINEL2/3,SENTINEL2/11,SERENUS_GEOTIFF/8,SERENUS_GEOTIFF/4,SERENUS_GEOTIFF/12,feature
0,"[0.1494, 0.1932, 0.1029, 0.2609, 0.188, 0.2016...","[0.1024, 0.1024, 0.1024, 0.1024, 0.1024, 0.102...","[0.1458, 0.146, 0.1451, 0.1451, 0.145, 0.1418,...","[0.1593, 0.1594, 0.1565, 0.1566, 0.1564, 0.147...","[-0.9757575988769531, -0.4324128031730652, -0....","[0.1739, 0.1738, 0.1728, 0.1728, 0.1725, 0.161...","[0.6143, 0.5683, 0.4452, 0.717, 0.7531, 0.7041...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1151, 0.1147, 0.1118, 0.1138, 0.1133, 0.109...","[0.5517, 0.5419, 0.4087, 0.7063, 0.7182, 0.662...","[0.0041, 0.0038, 0.0036, 0.0029, 0.0022, 0.002...","[-0.9389830231666565, -0.939830482006073, -0.9...","[0.5838, 0.5501, 0.4258, 0.7178, 0.7346, 0.685...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.06421815603971481, 0.08848532289266586, 0.0...","[0.1198, 0.1196, 0.1195, 0.1197, 0.1195, 0.115...","[0.5842, 0.5582, 0.4278, 0.7326, 0.766, 0.7011...","[0.596, 0.5604, 0.4305, 0.7328, 0.764, 0.7021,...","[0.0072, 0.0071, 0.0071, 0.0071, 0.007, 0.0068...","[0.6363, 0.6005, 0.4758, 0.7421, 0.7738, 0.729...","[0.159, 0.159, 0.1599, 0.1604, 0.1605, 0.1499,...","[0.2034602016210556, 0.20485268533229828, 0.21...","[0.1455, 0.1456, 0.1417, 0.1419, 0.1416, 0.130...","[0.29049816727638245, 0.2924862504005432, 0.30...","[0.5794, 0.5385, 0.4123, 0.7041, 0.7313, 0.679...","[6.5535, 6.5535, 6.5535, 6.5535, 6.5535, 6.553...",[b'corn'],"[0.1488, 0.3585, 0.2061, 0.5635, 0.3848, 0.260...","[0.5874, 0.5504, 0.43, 0.7209, 0.7484, 0.6918,...","[0.1956, 0.1971, 0.1029, 0.2548, 0.1756, 0.281...","[0.1979, 0.1978, 0.1937, 0.1939, 0.1938, 0.186...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1278, 0.129, 0.1277, 0.1288, 0.1284, 0.1283...","[0.044972386211156845, 0.06196702644228935, 0....","[0.5447, 0.495, 0.3897, 0.6618, 0.6777, 0.6333...","[0.0024, 0.0781, 0.0259, 0.3199, 0.0821, 0.002...","[0.1847, 0.1821, 0.1834, 0.1832, 0.1828, 0.180...","[0.1298, 0.1279, 0.1282, 0.1309, 0.1305, 0.125...","[0.2288, 0.2289, 0.2299, 0.2308, 0.231, 0.2178...","[0.1494, 0.1932, 0.1029, 0.2609, 0.188, 0.2016..."
1,"[0.4206, 0.337, 0.1363, 0.2079, 0.2924, 0.1472...","[0.1024, 0.1024, 0.0, 0.1024, 0.1024, 0.1024, ...","[0.1589, 0.1872, 0.1904, 0.1907, 0.1871, 0.162...","[0.1121, 0.2188, 0.2386, 0.1698, 0.1577, 0.155...","[-0.984599769115448, -0.9402085542678833, -0.9...","[0.1187, 0.2574, 0.2905, 0.1765, 0.192, 0.1906...","[0.5388, 0.4972, 0.3307, 0.6163, 0.7032, 0.687...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.104, 0.2942, 0.1511, 0.1257, 0.1299, 0.1219...","[0.5403, 0.5158, 0.375, 0.5867, 0.6629, 0.643,...","[0.0006, 0.1419, 0.1393, 0.1364, 0.1345, 0.131...","[-0.9636576771736145, -0.9060665369033813, -0....","[0.5373, 0.4184, 0.3505, 0.6138, 0.6712, 0.662...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.11580117046833038, -0.0443173311650753, 0.1...","[0.0749, 0.0859, 0.1043, 0.1133, 0.1129, 0.096...","[0.5821, 0.4032, 0.4009, 0.6028, 0.7132, 0.685...","[0.5659, 0.3965, 0.3727, 0.6196, 0.7077, 0.689...","[0.0031, 0.0072, 0.0068, 0.0073, 0.0074, 0.006...","[0.5581, 0.3875, 0.3305, 0.6331, 0.7438, 0.721...","[0.1306, 0.0872, 0.0935, 0.1036, 0.1089, 0.116...","[0.0660080835223198, -0.06671500951051712, 0.3...","[0.1063, 0.206, 0.2207, 0.1395, 0.1327, 0.1383...","[0.0942414328455925,

In [35]:
final_cols = ["golden_label", "feature"]

In [36]:
df = df[final_cols]

In [37]:
df.head()

,golden_label,feature
0,[b'corn'],"[0.1494, 0.1932, 0.1029, 0.2609, 0.188, 0.2016..."
1,[b'corn'],"[0.4206, 0.337, 0.1363, 0.2079, 0.2924, 0.1472..."
2,[b'others'],"[0.2912, 0.2297, 0.2478, 0.2295, 0.1891, 0.279..."
3,[b'fallow'],"[0.3085, 0.1232, 0.3373, 0.2428, 0.1208, 0.276..."
4,[b'drybean'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [52]:
df1 = df[df.golden_label.isin(["[b'grassland']", "[b'wheat']"])]
df1 = df1.groupby("golden_label").apply(lambda group: group.head(2+1)).reset_index(drop="first")

In [53]:
df1

,golden_label,feature
0,[b'grassland'],"[0.1826, 0.1889, 0.1702, 0.2938, 0.1801, 0.169..."
1,[b'grassland'],"[0.119, 0.1609, 0.1492, 0.1726, 0.4192, 0.1061..."
2,[b'grassland'],"[0.1388, 0.1161, 0.0924, 0.3727, 0.3812, 0.352..."
3,[b'wheat'],"[0.3549, 0.2354, 0.2218, 0.0978, 0.2333, 0.255..."
4,[b'wheat'],"[0.2878, 0.1821, 0.2575, 0.1837, 0.115, 0.1872..."
5,[b'wheat'],"[0.2046, 0.2984, 0.1472, 0.2375, 0.4454, 0.129..."


In [62]:
n = 2
k = 3

In [63]:
index_ = [i + k*j for i in range(k) for j in range(n)]

In [64]:
index_

[0, 3, 1, 4, 2, 5]

In [65]:
df1.reindex(index_)

,golden_label,feature
0,[b'grassland'],"[0.1826, 0.1889, 0.1702, 0.2938, 0.1801, 0.169..."
3,[b'wheat'],"[0.3549, 0.2354, 0.2218, 0.0978, 0.2333, 0.255..."
1,[b'grassland'],"[0.119, 0.1609, 0.1492, 0.1726, 0.4192, 0.1061..."
4,[b'wheat'],"[0.2878, 0.1821, 0.2575, 0.1837, 0.115, 0.1872..."
2,[b'grassland'],"[0.1388, 0.1161, 0.0924, 0.3727, 0.3812, 0.352..."
5,[b'wheat'],"[0.2046, 0.2984, 0.1472, 0.2375, 0.4454, 0.129..."


In [41]:
df1["feature"].tolist()

[[0.1826,
  0.1889,
  0.1702,
  0.2938,
  0.1801,
  0.1697,
  0.4276,
  0.2909,
  0.3,
  0.1995,
  0.3175,
  0.1236,
  0.2174,
  0.1925,
  0.1867,
  0.1688,
  0.2601,
  0.1823,
  0.3791,
  0.1535,
  0.1186,
  0.1453,
  0.3419,
  0.1638,
  0.1356,
  0.2398,
  0.1235,
  0.0283,
  0.1154,
  0.1244,
  0.109,
  0.1219,
  0.4422,
  0.1158,
  0.1015,
  0.2289,
  0.2195,
  0.4012,
  0.086,
  0.0861,
  0.061,
  0.1532,
  0.235,
  0.0859,
  0.0957,
  0.1719,
  0.0816,
  0.0937,
  0.1318,
  0.0767,
  0.0795,
  0.0106,
  0.1355,
  0.0801,
  0.1169,
  0.119,
  0.0921,
  0.114,
  0.108,
  0.1073,
  0.0868,
  0.121,
  0.0927,
  0.3107,
  0.3107,
  0.1379,
  0.1369,
  0.1504,
  0.2199,
  0.1631,
  0.2171,
  0.1586,
  0.1856,
  0.3367,
  0.3367,
  0.2048,
  0.0,
  0.1024,
  0.1024,
  0.0,
  0.1024,
  0.1024,
  0.1024,
  0.1024,
  0.0,
  0.1024,
  0.2048,
  0.0,
  0.0,
  0.2048,
  0.2048,
  0.1024,
  0.0,
  0.1024,
  0.0,
  0.2048,
  0.0,
  0.1024,
  0.0,
  0.0,
  0.1024,
  0.0,
  0.1024,
  0.0,
  0.0,


In [199]:
len(df["feature"][1])

2925

In [ ]:
def get_images(paths, labels, nb_samples=None, shuffle=True):
    """
    Takes a set of character folders and labels and returns paths to image files
    paired with labels.
    Args:
        paths: A list of character folders
        labels: List or numpy array of same length as paths
        nb_samples: Number of images to retrieve per character
    Returns:
        List of (label, image_path) tuples
    """
    if nb_samples is not None:
        sampler = lambda x: random.sample(x, nb_samples)
    else:
        sampler = lambda x: x
    images_labels = [
        (i, os.path.join(path, image))
        for i, path in zip(labels, paths)
        for image in sampler(os.listdir(path))
    ]
    if shuffle:
        random.shuffle(images_labels)
    return images_labels


In [ ]:
class DataGenerator(IterableDataset):
    """
    Data Generator capable of generating batches of Omniglot data.
    A "class" is considered a class of omniglot digits.
    """

    def __init__(
        self,
        num_classes,
        num_samples_per_class,
        batch_type,
        config={},
        device=torch.device("cpu"),
        cache=True,
        df=df
    ):
        """
        Args:
            num_classes: Number of classes for classification (N-way)
            num_samples_per_class: num samples to generate per class in one batch (K+1)
            batch_size: size of meta batch size (e.g. number of functions)
            batch_type: train/val/test
        """
        self.num_samples_per_class = num_samples_per_class
        self.num_classes = num_classes

        self.dim_input = len(df["feature"][0])
        self.dim_output = self.num_classes


        field_df = df

        random.seed(1)
        field_df = field_df.sample(frac=1).reset_index(drop=True)
        num_val = 100
        num_train = 1100
        self.metatrain_character_folders = field_df[:num_train]
        self.metaval_character_folders = field_df[num_train : num_train + num_val]
        self.metatest_character_folders = field_df[num_train + num_val :]
        self.device = device
        self.image_caching = cache
        self.stored_images = {}
        self.field_list = field_df["golden_label"].unique().tolist()

        if batch_type == "train":
            self.folders = self.metatrain_character_folders
        elif batch_type == "val":
            self.folders = self.metaval_character_folders
        else:
            self.folders = self.metatest_character_folders

    def _sample(self):
        """
        Samples a batch for training, validation, or testing
        Args:
            does not take any arguments
        Returns:
            A tuple of (1) Image batch and (2) Label batch:
                1. image batch has shape [K+1, N, 2925] and
                2. label batch has shape [K+1, N, N]
            where K is the number of "shots", N is number of classes
        Note:
            1. The numpy functions np.random.shuffle and np.eye (for creating)
            one-hot vectors would be useful.

            2. For shuffling, remember to make sure images and labels are shuffled
            in the same order, otherwise the one-to-one mapping between images
            and labels may get messed up. Hint: there is a clever way to use
            np.random.shuffle here.
            
            3. The value for `self.num_samples_per_class` will be set to K+1 
            since for K-shot classification you need to sample K supports and 
            1 query.
        """

        #############################
        #### YOUR CODE GOES HERE ####
        sample_field = random.sample(self.field_list, self.num_classes)
        labels = np.eye(self.num_classes)
        k = self.num_samples_per_class-1
        sample_set = get_images(sample_field, labels, nb_samples=k, shuffle=False)
        query_set = get_images(sample_field, labels, nb_samples=1, shuffle=True)
        sample_set.extend(query_set)

        images = []
        labels = []
        for label, image_file in sample_set:
            labels.append(label)    
            images.append(self.image_file_to_array(image_file, self.dim_input))

        images = torch.from_numpy(np.array(images)).float()
        labels = torch.from_numpy(np.array(labels)).long()
        images_tensor = images.reshape((self.num_samples_per_class, self.num_classes, self.dim_input))
        labels_tensor = labels.reshape((self.num_samples_per_class, self.num_classes, self.num_classes))
        assert(images_tensor.shape==(self.num_samples_per_class, self.num_classes, self.dim_input))
        assert(labels_tensor.shape==(self.num_samples_per_class, self.num_classes, self.num_classes))

        return (images_tensor, labels_tensor)
        #############################

    def __iter__(self):
        while True:
            yield self._sample()

In [183]:
df.head(3)

,golden_label,feature
0,[b'corn'],"[0.1494, 0.1932, 0.1029, 0.2609, 0.188, 0.2016..."
1,[b'corn'],"[0.4206, 0.337, 0.1363, 0.2079, 0.2924, 0.1472..."
2,[b'others'],"[0.2912, 0.2297, 0.2478, 0.2295, 0.1891, 0.279..."


In [187]:
field_df = df.sample(frac=1).reset_index(drop=True)

In [189]:
df.head(3)

,golden_label,feature
0,[b'corn'],"[0.1494, 0.1932, 0.1029, 0.2609, 0.188, 0.2016..."
1,[b'corn'],"[0.4206, 0.337, 0.1363, 0.2079, 0.2924, 0.1472..."
2,[b'others'],"[0.2912, 0.2297, 0.2478, 0.2295, 0.1891, 0.279..."


In [188]:
field_df.head(3)

,golden_label,feature
0,[b'cotton'],"[0.1477, 0.1358, 0.1771, 0.3279, 0.0345, 0.147..."
1,[b'corn'],"[0.2351, 0.3165, 0.2436, 0.0394, 0.324, 0.2223..."
2,[b'soybean'],"[0.2351, 0.1948, 0.0796, 0.1136, 0.3872, 0.443..."


In [194]:
field_list = field_df["golden_label"].unique().tolist()

In [195]:
field_list

["[b'cotton']",
 "[b'corn']",
 "[b'soybean']",
 "[b'wheat']",
 "[b'sorghum']",
 "[b'grassland']",
 "[b'fallow']",
 "[b'drybean']",
 "[b'canola']",
 "[b'others']",
 "[b'barley']",
 "[b'alfalfa']"]

In [196]:
random.sample(field_list, 3)

["[b'barley']", "[b'wheat']", "[b'grassland']"]

In [167]:
def initialize_weights(model):
    if type(model) in [nn.Linear]:
        nn.init.xavier_uniform_(model.weight)
        nn.init.zeros_(model.bias)
    elif type(model) in [nn.LSTM, nn.RNN, nn.GRU]:
        nn.init.orthogonal_(model.weight_hh_l0)
        nn.init.xavier_uniform_(model.weight_ih_l0)
        nn.init.zeros_(model.bias_hh_l0)
        nn.init.zeros_(model.bias_ih_l0)

In [172]:
class MANN(nn.Module):
    def __init__(self, num_classes, samples_per_class, hidden_dim):
        super(MANN, self).__init__()
        self.num_classes = num_classes
        self.samples_per_class = samples_per_class

        self.layer1 = torch.nn.LSTM(num_classes + 784, hidden_dim, batch_first=True)
        self.layer2 = torch.nn.LSTM(hidden_dim, num_classes, batch_first=True)
        initialize_weights(self.layer1)
        initialize_weights(self.layer2)

    def forward(self, input_images, input_labels):
        """
        MANN
        Args:
            input_images: [B, K+1, N, 784] flattened images
            labels: [B, K+1, N, N] ground truth labels
        Returns:
            [B, K+1, N, N] predictions
        """
        #############################
        #### YOUR CODE GOES HERE ####        
        B, K, N, I = input_images.shape[0], input_images.shape[1]-1,\
                     input_images.shape[2], input_images.shape[3]
        labels = torch.clone(input_labels)
        labels[:, K, :] = 0  # set the query set label to be 0 
        x = torch.cat((input_images, labels), axis=3) 
        x = x.reshape(B, (K+1)*N, N+I) 
        x, _ = self.layer1(x)
        x, _ = self.layer2(x)        
        x = x.reshape(B, K+1, N, N)
        return x
        #############################

    def loss_function(self, preds, labels):
        """
        Computes MANN loss
        Args:
            preds: [B, K+1, N, N] network output
            labels: [B, K+1, N, N] labels
        Returns:
            scalar loss
        Note:
            Loss should only be calculated on the N test images
        """
        #############################
        #### YOUR CODE GOES HERE ####       
        labels_ = torch.clone(labels)
        preds = preds[:, -1:].squeeze(1).reshape(-1, self.num_classes)
        labels_ = labels_[:, -1:].squeeze(1).reshape(-1, self.num_classes).argmax(axis=1)
        return nn.CrossEntropyLoss()(preds, labels_)
        #############################